## Funciones útiles


In [2]:
import numpy as np 
from numpy import linalg as LA

def Grad(f, x0, h=1e-6, i=-1):
    """
    Función que calcula el Grad de una función en un punto
    """
    n = len(x0)
    if i in range(n):
        z = np.zeros(n)
        z[i] = h/2
        Grad = (f(x0 + z) - f(x0 - z))/h
    else:
        Grad = np.zeros(n)
        for j in range(n):
            z = np.zeros(n)
            z[j] = h/2
            Grad[j] = (f(x0 + z) - f(x0 - z))/h
    return np.array(Grad)


def Hess(f, x0, h=1e-4, method="basic"):
    """
    Función que calcula la Hessiana  de una función en un punto.
    f: función sobre la cual queremos calcular la hessiana.
    x0: Punto sobre el cual queremos hacer el cálculo
    h: nivel de precisión para hacer el cálculo
    method: Método por el cual se quiere hacer puede ser:
             'basic', 'grad', 'centered', 'gradCentered'
    """
    n = len(x0)
    Hess = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            z_i = np.zeros(n)
            z_i[i] = h
            z_j = np.zeros(n)
            z_j[j] = h
            if method == "basic":
                Hess[i, j] = (f(x0 + z_j + z_i) - f(x0 + z_i) -
                              f(x0+z_j) + f(x0)) / (h**2)
            elif method == "grad":
                Hess[i, j] = (Grad(f, x0+z_j, h, i) - Grad(f, x0, h, i) +
                              Grad(f, x0+z_i, h, j) - Grad(f, x0, h, j))/(2*h)
            elif method == "centered":
                if i == j:
                    Hess[i, j] = (-f(x0+2*z_i) + 16*f(x0+z_i) - 30*f(x0) +
                                  16*f(x0-z_i) - f(x0-2*z_i)) / (12*h**2)
                else:
                    Hess[i, j] = (f(x0+z_i+z_j) - f(x0 + z_i - z_j) -
                                  f(x0 - z_i + z_j) + f(x0-z_i-z_j))/(4*h**2)
            elif method == "gradCentered":
                Hess[i, j] = (Grad(f, x0+z_j, h)[i] - Grad(f, x0-z_j, h)[i] +
                              Grad(f, x0+z_i, h)[j] - Grad(f, x0-z_i, h)[j])\
                               / (4 * h)
    return Hess

def condiciones_wolfe(f, x0, alpha, pk, c1=1e-4, c2=1e-2, tol=1e-5):
    """
    Función que evalúa las condiciones de wolfe para una alpha.
    f:  función que optimizamos
    x0: punto anterior un numpy.array
    alpha: valor que cumplirá condiciones de wolfe.
    pk: dirección de decenso un numpy.array
    """
    def grad(alpha): return Grad(f, x0+alpha*pk, tol)
    def phi(alpha): return f(x0 + alpha*pk)  # Ojo que phi(0) = f(x0)
    def linea(alpha): return phi(0) + c1 * alpha * np.dot(g_x0, pk)
    g_x0 = grad(0)  # grad(0) = Grad(f,x0)
    cond_1 = linea(alpha) - phi(alpha) >= 0
    cond_2 = np.dot(grad(alpha), pk) - c2 * np.dot(g_x0, pk) >= 0
    return cond_1 and cond_2


def genera_alpha(f, x0, pk, c1=1e-4, c2 = 0.5, tol=1e-5):
    """
    Backtracking LS i.e. Algoritmo que encuentra una
    alpha que cumpla condiciones de wolfe.
    """
    alpha, rho = 1, 3/4
    Gkpk = Grad(f, x0).dot(pk)
    while f(x0 + alpha*pk) > f(x0) + c1*alpha*Gkpk:
        alpha *= rho
    return alpha

def cuadrados(x):
    resultado = 0
    for i in range(len(x)):
        resultado += x[i]**2
    return resultado



## 1. BFGS con Xo dada con actualizacion DFP


In [15]:
def DFP_Bk(yk, sk, Bk):
    """
    Función que calcula La actualización DFP de la matriz Bk
    In:
      yk: Vector n
      sk: Vector n
      Bk: Matriz nxn
    Out:
      Bk+1: Matriz nxn
    """
    n = len(yk)
    rhok = 1 / (yk.T*sk)
    Vk = (np.eye(n) - rhok * yk*sk.T)
    Bk1 = Vk * Bk * Vk + rhok * yk * yk.T
    return Bk1


def DFP_Hk(yk, sk, Hk):
    """
    Función que calcula La actualización DFP de la matriz Hk
    In:
      yk: Vector n
      sk: Vector n
      Hk: Matriz nxn
    Out:
      Hk+1: Matriz nxn
    """
    yk = np.array([yk]).T
    sk = np.array([sk])
    Hk1 = Hk - (Hk*(yk.dot(yk.T))* Hk)/((yk.T.dot(Hk)).dot(yk)) + (sk*sk.T)/(yk.T.dot(sk.T))
    return Hk1


def BFGS_Hk(yk, sk, Hk):
    """
    Función que calcula La actualización BFGS de la matriz Hk
    In:
      yk: Vector n
      sk: Vector n
      Hk: Matriz nxn
    Out:
      Hk+1: Matriz nxn
    """
    n = len(yk)
    yk = np.array([yk]).T
    sk = np.array([sk]).T
    rhok = 1 / yk.T.dot(sk)
    Vk = (np.eye(n) - rhok * yk.dot(sk.T))
    Hk1 = Vk.T * Hk * Vk + rhok * sk.dot(sk.T)
    return Hk1


def BFGS_Bk(yk, sk, Bk):
    """
    Función que calcula La actualización BFGS de la matriz Bk
    In:
      yk: Vector n
      sk: Vector n
      Bk: Matriz nxn
    Out:
      Bk+1: Matriz nxn
    """
    return Bk - (np.dot(Bk, np.dot(sk, np.dot(sk, Bk)))) / (np.dot(sk, np.dot(Bk, sk))) + np.dot(yk, yk) / np.dot(yk, sk)

def BFGS(f, x0, tol, H0, maxiter=10000):
    k = 0
    Gk = Grad(f, x0)
    Hk = H0
    xk = np.array(x0)
    xk1 = np.array(x0)
    sk = np.array(100)
    while (LA.norm(Gk) > tol and LA.norm(sk) > tol and k <= maxiter):
        pk = - Hk.dot(Gk)
        alphak = genera_alpha(f, xk, pk)
        xk1 = xk + alphak * pk
        sk = xk1 - xk
        Gk1 = Grad(f, xk1)
        yk = Gk1 - Gk
        Hk = DFP_Hk(yk, sk, Hk)
        k += 1
        xk = xk1
        Gk = Gk1
    return xk1, k


if __name__ == "__main__":
    x, k = BFGS(cuadrados,[(-1)**i*10 for i in range(10)] , 1e-15, np.eye(10))
    print(f'Llegué a {x} en {k} iteraciones')


Llegué a [-1.72317626e-12  1.72325892e-12 -1.27831007e-12  1.27891189e-12
 -9.41696127e-13 -1.41963461e-12  2.52426391e-12 -2.43234157e-12
  2.60964362e-12  2.04190061e-12] en 795 iteraciones


## 2. Gradiente Conjugado 


In [4]:
def gradiente_conjugado_MOD(x0, A, b):
    xk = x0
    b = np.matrix(b).T
    c = [0 for i in range(len(a[2]))]
    for i in range(len(A[2])):
        c[i] = x0[i]*A[2][i]
    c = np.matrix(c)
    rk = c.T - b
    pk = -rk
    d = [0 for i in range(len(A[2]))]
    while not (rk.T * rk ==  0):
        for i in range(len(A[2])):
            d[i] = pk[i]*A[2][i]
        alphak = rk.T * rk /(np.dot(pk,d))
        alphak= alphak[0,0]
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * A * pk
        betak_1 = (rk_1.T * rk_1) / (rk.T * rk)
        betak_1 = betak_1[0,0]
        pk_1 = -rk_1 + betak_1 * pk
        xk, rk, pk = xk_1, rk_1, pk_1
    return xk

def Matriz_rala(z):
    n = len(z)
    Resumida = [[0 for i in range(n)] for j in range(3)]
    for i in range(n):
        Resumida[0][i] = i
        Resumida[1][i] = i
        Resumida[2][i] = z[i]
    return Resumida
                   
import random
random.seed(174139) #  Cambien a su propia clave
Diag_A = [random.randint(1,1000) for x in range(1000000)]

# Resolvemos entonces el problema
n=1000000
A = Matriz_Rala(Diag_A)
b = [random.randint(1,1000) for x in range(1000000)]
x0 = np.matrix(np.zeros(n)).T
print(gradiente_conjugado_MOD(x0, A, b))

KeyboardInterrupt: 